In [20]:
from langchain_groq import ChatGroq

In [21]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [22]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/jobs?filter%5Bcategory%5D%5B0%5D=Technology")
page_data = loader.load().pop().page_content
print(page_data)






















Search Open Nike Jobs








































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Language


In [33]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [34]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Associate Audio Visual Specialist, Converse Technology',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Director – Global Tax Technology Operations',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Director, Solution Validation Engineering',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Director, Technical Product Management, SEC, Finance (Remote Work Option)',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Director, Technical Project Management',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Junior Software Engineer',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Lead Cloud Engineer, Converse Technology (Open to Atlanta, Beaverton, Boston location)',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Lead Content Activation Engineer',
  'experience': None,
  'skills': None,
  'descri

In [35]:
type(json_res)

list

In [44]:
import pandas as pd

df = pd.read_csv(r"C:\Users\vkush\cold email generator\App\Resource\my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [52]:
import uuid
import chromadb
import pandas as pd

# Assuming df is your DataFrame containing the data
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

# Add data to collection if it's empty
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=[row["Techstack"]],  # Document field for querying
                       metadatas={"links": row["Links"]},  # Metadata containing links
                       ids=[str(uuid.uuid4())])

# Loop through the jobs and query based on skills
for job_item in job:
    skills = job_item.get('skills', '')  # Extract skills, default to empty string if not found
    query_result = collection.query(query_texts=[skills], n_results=2)
    
    # Get the metadata (links) associated with the best matches
    links = query_result.get('metadatas', [])
    print(links)


ValueError: Expected document to be a str, got None in query.

In [49]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

ValueError: At least one of one of embeddings, documents, images, uris must be provided in query.

In [16]:
job

NameError: name 'job' is not defined

In [17]:
job = json_res
job['skills']

TypeError: list indices must be integers or slices, not str

In [16]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert AI/ML Solutions for Nike's Data Science Services and Platform

Dear Hiring Manager,

I came across the job description for a Software Engineer II, AI/ML Platforms at Nike, and I'm excited to introduce AtliQ, a leading AI & Software Consulting company that can help you design and implement core components of your data science services and platform.

With our expertise in AI, ML, and software development, we can help you scale your platform to serve Nike globally. Our team of experts has a strong foundation in computer science and experience with modern programming languages such as Java, Python, and Golang. We're well-versed in Infrastructure as Code / DevOps practices and technologies like Terraform, and have hands-on experience with Cloud services like AWS and Azure.

Our portfolio showcases our capabilities in developing and deploying scalable, performant, and reliable solutions. I'd like to highlight a few relevant examples:

* Our Machine Learning with Python portfo